In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

2025-04-08 09:01:52.557216: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import pandas_ta as ta

In [14]:
market =pd.read_csv('../data/processed-data/price-gold.csv', index_col='Date',  parse_dates=True)
market.drop('Change %', axis=1, inplace=True)
market.head(20)

,Close,Open,High,Low,Vol.
Date,,,,,
2015-01-02,1186.2,1184.0,1194.9,1167.3,120680.0
2015-01-05,1204.0,1187.8,1207.5,1177.8,149110.0
2015-01-06,1219.4,1204.0,1223.3,1201.6,190360.0
2015-01-07,1210.7,1219.0,1219.4,1209.1,117090.0
2015-01-08,1208.5,1211.0,1216.8,1204.2,122490.0
2015-01-09,1216.1,1208.9,1224.0,1207.0,155980.0
2015-01-12,1232.8,1223.0,1236.0,1217.5,138520.0
2015-01-13,1234.4,1233.1,1244.5,1227.4,179650.0
2015-01-14,1234.5,1231.2,1244.6,1224.9,193110.0


In [72]:
atr_period = 14
market["ATR"] = ta.atr(market["High"], market["Low"], market["Close"], length=atr_period)
market.head(20)

,Close,Open,High,Low,Vol.,ATR
Date,,,,,,
2015-01-02,1186.2,1184.0,1194.9,1167.3,120680.0,NaN
2015-01-05,1204.0,1187.8,1207.5,1177.8,149110.0,NaN
2015-01-06,1219.4,1204.0,1223.3,1201.6,190360.0,NaN
2015-01-07,1210.7,1219.0,1219.4,1209.1,117090.0,NaN
2015-01-08,1208.5,1211.0,1216.8,1204.2,122490.0,NaN
2015-01-09,1216.1,1208.9,1224.0,1207.0,155980.0,NaN
2015-01-12,1232.8,1223.0,1236.0,1217.5,138520.0,NaN
2015-01-13,1234.4,1233.1,1244.5,1227.4,179650.0,NaN
2015-01-14,1234.5,1231.2,1244.6,1224.9,193110.0,NaN


In [16]:
def create_sequences(data, lookback):
        X, y = [], []
        for i in range(lookback, len(data)-6):
            X.append(data[i - lookback:i])
            y.append(data[i:i+6, 1])  # Predict the 'Close' price (index 1)
        return np.array(X), np.array(y)

In [17]:
from joblib import load

model = tf.keras.models.load_model('../models/model1/lstm_model.keras')
scaler = load('../models/model1/scaler.save')

/home/elm19/miniconda3/envs/goldspot-predictor/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:

class trade:
    def __init__(self, market, model, scaler, lookback_period=10):
        self.market = market
        self.data = market[['Open', 'Close', 'High', 'Low', 'Vol.']].copy()
        self.scaled_data = scaler.transform(self.data)
        self.model = model
        self.scaler = scaler
        self.lookback_period = lookback_period
        self.X, self.y = create_sequences(self.scaled_data, lookback_period)

    def inverse_transform(self, data):
        # Inverse transform the predictions
        predicted_inv = np.zeros((data.shape[0], data.shape[1]))
        for i in range(data.shape[1]):
            dummy_predictions = np.zeros((len(data), 5))
            dummy_predictions[:, 1] = data[:, i]
            predicted_inv[:, i] = scaler.inverse_transform(dummy_predictions)[:, 1]


        return predicted_inv
    
    def backtest(self):
        X_backtest = np.reshape(self.X, (self.X.shape[0], self.X.shape[1], self.X.shape[2]))
        predictions = self.model.predict(X_backtest)
        # Inverse transform the predictions
        predictions = self.inverse_transform(predictions)

        reely = self.inverse_transform(self.y)

        buy, sell, hold = [0, 0], [0, 0], [0, 0]
        trade = [0, 0]
        for (i, prediction) in enumerate(predictions):
            
            if i < max(atr_period, self.lookback_period):
                continue
            atr = self.market["ATR"].iloc[i]
            price = self.market["Close"].iloc[i]
            if(max(prediction - price) > 1.5 * atr):
                # print(f"Buy signal at index {i}, prediction: {prediction}")
                buy[0] += 1
                trade[0] = i
                if(max(reely[i] -price) > 1.5 * atr):
                    buy[1] += 1
               

            elif(min(prediction - price) < -1.5* atr):
                # print(f"Sell signal at index {i}, prediction: {prediction}")
                sell[0] += 1
                if(min(reely[i] -price) < -1.5 * atr):
                    sell[1] += 1
            else:
                # print(f"Hold signal at index {i}, prediction: {prediction}")
                hold[0] += 1
                if(abs(min(reely[i]) - price) < 1.5 * atr) and (abs(max(reely[i]) - price) < 1.5 * atr):
                    hold[1] += 1

        print(f"Total Buy signals: {buy}")
        print(f"Total Sell signals: {sell}")
        print(f"Total Hold signals: {hold}")


In [78]:

trade_instance = trade(market, model, scaler, lookback_period=10)
trade_instance.backtest()

72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Total Buy signals: [982, 727]
Total Sell signals: [273, 266]
Total Hold signals: [1031, 445]
